<a href="https://colab.research.google.com/github/salarbalou/Photothermal_Study_CQDs/blob/main/Photothermal_Study_of_CQDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import numpy as np
import plotly.io as pio
pio.templates.default= 'plotly_dark'
from plotly.subplots import make_subplots

In [157]:
colors = ['red', 'cyan']
concentration = '0.75'
dcs = '20', '40', '60', '80', '100'
space = '_'
baseurl = 'https://raw.githubusercontent.com/salarbalou/Photothermal_Study_CQDs/main/'
format = '.txt'

In [211]:
fig = make_subplots(rows = 1, cols = 2)
#for blue 435nm maxpower=4W laser:
m, b = 7.025524475524473, -66.07575757575763
# dc is arduino board pwm pin duty cycle
water_dcs = '20', '40', '60', '80', '100'
HRs  = [] #list of heating rates
CRs = [] #list of cooling rates
experiments = []
for water_dc in water_dcs:
 data_water = np.loadtxt(baseurl+'water'+space+'1'+space+water_dc+format, delimiter=' , ')[9:]
 Iout_w= m*np.float(water_dc) + b
 twh, Twh = data_water[:,0][0:121], data_water[:,1][0:121]
 twc, Twc = data_water[:,0][122::], data_water[:,1][122::]
 fig.add_scatter(x  = twh, y = Twh, name = f'Heating water at {np.int(Iout_w)} mW', row = 1, col =1, marker_color = 'white')
 fig.add_scatter(x  = twc, y = Twc, name = f'Cooling water at {np.int(Iout_w)} mW', row = 1, col =2, marker_color = 'white')
 fig.update_xaxes(nticks = 20, tickfont = dict(size = 18), title = 'time (seconds)', title_font = dict(size = 18))
 fig.update_yaxes(nticks = 20, tickfont = dict(size = 18), title = 'Temperature (˚C)', title_font = dict(size = 18))
for color in colors:
  for dc in dcs:
     data = np.loadtxt(baseurl+color+space+concentration+space+dc+format, delimiter=' , ')[9:]
     Iout= m*np.float(dc) + b
     # *** Steady State Analysis *** Heating rate **** #turning on laser for 120 seconds to measure the heating rate
     th = data[:, 0][0:121] #time (s)
     tc = data[:,0][122::]
     Th = data[:, 1][0:121] #temperature of the solution (C)
     # *** Steady State Analysis *** Cooling rate **** #turning off laser for 120 seconds to measure the cooling rate
     Tc = data[:,1][122::]
     Rh  = np.gradient(Th, th) #gradient of T with respect to time at max which is considered the heating rate
     Rc = np.gradient(Tc, tc) #gradient of T with respect to time when laser was off corresponding to the cooling rate
     HR = np.max(Rh) #heating rate
     CR = np.min(Rc) #cooling rate
     Teqh = Th[np.where(Rh <= np.min(R))[0]] #equilibrium temperature (C)
     teqh = th[np.where(Rh <= np.min(R))[0]] # time required to reach equilibrium (s)
     experiments.append(f'{color} CQD {concentration}x at {np.int(Iout)} mW (dutycycle:{dc}) with Heating rate of {np.round(HR, 4)} (˚C/s) and Cooling rate of {np.round(-CR, 4)} (˚C/s)')
     fig.add_scatter(x= th, y = Th, name = f'Heating {color} CQD {concentration}x at {np.int(Iout)} mW', mode = 'markers+lines', marker_color = color, row =1, col = 1)
     fig.add_scatter(x= tc, y = Tc, name = f'Cooling {color} CQD {concentration}x at {np.int(Iout)} mW', mode = 'markers+lines', marker_color = color, row =1, col = 2)
     fig.update_layout(height  = 800, width = 2000, title =  'CQD Heating and Cooling with Laser (430nm)')
     fig.update_xaxes(nticks = 20, tickfont = dict(size = 18), title = 'time (seconds)', title_font = dict(size = 18))
     fig.update_yaxes(nticks = 25, tickfont = dict(size = 18), title = 'Temperature (˚C)', title_font = dict(size = 18)) 
     HRs.append(HR)
fig.show()

In [212]:
# Which of the samples at which concentartion and duty cycle is the optimized version
optimized_sample = experiments[np.argmax(HRs)]
print(optimized_sample)

red CQD 0.75x at 636 mW (dutycycle:100) with Heating rate of 10.075 (˚C/s) and Cooling rate of 4.59 (˚C/s)


In [166]:
# *** Photothermal conversion efficiency ***

#Iout = #measured using powermeter: f(CCQD, L_laser) (W)
#S_PCR_tube = #surface area of the solution at which laser beam is passing through:[PCR_tube, paper] (m**2)
#S_paper = #surface are of the paper at which laser beam is covering:[PCR_tube, paper]  (m**2)
#m = rho * V #mass of solution (Kg)

Iin = m*dc + b #input power of the laser: f(dc) (W)
h = 1000 #heat transfor coefficient of solution with its surrounding (W/(m**2)*C)

rho  = 1000 #density of solution (kg/m**3)
V = 1e-8 #volume of the solution (m**3)
Cp = 4182 #heat capacity of water (J/Kg*C)
#eta = ((R*(Cp*m)+np.sum((h*S*(T - Tw))))/(Iin - Iout)
#print(f'Photothermal Conversion efficiency of your CQD is {eta}')

TypeError: ignored

In [ ]:
# ***Modeling the heat diffusion in PCR-Tube and paper***
nx = 2
x = np.arange(nx)
nt = np.int(teq) #temperature rise is evaluated in {t.size} seconds
Tplot = np.full(T.size, np.min(Tw))

In [ ]:
fig = make_subplots()
fig.add_scatter(x = x, y = Tplot, name = '0 s')
for i in range (nt):
  Tplot[0] = T[i]
  Tplot[-1] = np.min(Tw)
  fig.add_scatter(x = np.arange(nx), y = Tplot, name = f'{i+1} s')
fig.update_layout(height = 800, width = 1100)
fig.show()